In [36]:
! pip install python-dotenv 
! pip install openai

You should consider upgrading via the '/Users/leftrightleft/Code/leftrightleft/smart-scan-experiments/venv/bin/python3 -m pip install --upgrade pip' command.
  Using cached openai-0.28.1-py3-none-any.whl (76 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached aiohttp-3.8.5-cp37-cp37m-macosx_10_9_x86_64.whl (362 kB)
  Using cached multidict-6.0.4-cp37-cp37m-macosx_10_9_x86_64.whl (28 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.3.3-cp37-cp37m-macosx_10_9_x86_64.whl (36 kB)
  Using cached asynctest-0.13.0-py3-none-any.whl (26 kB)
  Using cached yarl-1.9.2-cp37-cp37m-macosx_10_9_x86_64.whl (64 kB)
You should consider upgrading via the '/Users/leftrightleft/Code/leftrightleft/smart-scan-experiments/venv/bin/python3 -m pip install --upgrade pip' command.


In [37]:
from dotenv import load_dotenv
import openai
import os
import json

### Assign vars

In [39]:
load_dotenv()
openai.api_key = os.environ.get('OPENAI_KEY')
model = 'gpt-3.5-turbo'
prompt = """You are a decision tool which decides whether or not a static analysis should occur using CodeQL on the following code diff.  
  You will decide yes for any changes to the operation of the application.  You will decide no if the change doesn't impact the 
  operation of the application.
  
  For example, a change that deletes a commented line of code would be a "NO".  A change that edits markdown in the repository 
  would be a "NO".  A change that adds a new variable would be a "YES". 
   
  The code snippet provided will be a diff from a GitHub pull request. You will analyze the diff and give a response of yes or no 
  along with an explanation.  
  
  Yes indicates a static analysis should occur, no indicates there are no changes that could be a security concern. Your response 
  should be formatted in json format.  "yes" or "no" will be in a key called "decision". The reason for the decision will be in a 
  key called "reason". Keep the reason to two or three sentences. 
  
  A response example: {"decision" : "no", "reason" : "You are editing a markdown file. This change doesn't impact the operation of your application"}'
"""



In [35]:
# Open files in diffs directory and read into a list
def read_files(dir):
    diffs_dir = dir
    diffs = []
    for filename in os.listdir(diffs_dir):
        with open(os.path.join(diffs_dir, filename), 'r') as f:
            diffs.append({"name":filename, "content":f.read()})
    return diffs

In [48]:
source_diffs = read_files('data/diffs_with_alerts')
for diff in source_diffs[:10]:
    if len(diff['content']) < 5000:
        completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": diff['content']},
        ],
        )
        response = json.loads(completion.choices[0].message["content"])
    else:
        response = {"decision" : "yes", "reason" : "The diff is too large to analyze"}
    
    print(f"filename: {diff['name']} response: {response['decision']} reason: {response['reason']}")

filename: octodemo-demo-bookstore-elizabethbarrord-11 response: yes reason: A new method 'getBooksByTitle' has been added to the BookDatabase interface and implemented in the BookDatabaseImpl class. This change could impact the operation of the application and should undergo static analysis.
filename: octodemo-elizabethbarrord-ghas-demo-8 response: no reason: This is a new workflow file added to the repository. It doesn't impact the operation of your application.
filename: octodemo-fmadotto-ghas-108 response: yes reason: The diff is too large to analyze
filename: octodemo-colind-bookstore-v5-11 response: yes reason: The code changes introduce a new method 'getBooksByTitle' in the 'BookDatabase' interface and its implementation in 'BookDatabaseImpl'. This change could impact the behavior of the application and should be analyzed for any security concerns.
filename: octodemo-lf-bookstore-20230925-5 response: yes reason: A new method 'getBooksByAuthor' has been added to the BookDatabase i

## Query OpenAI